## Perform machine learning on something close to real data

I'm going to perform using a home credit dataset from kaggle

#### Problem 1
Confirmation of competition contents

- **What to learn**: The transaction information of the clients
- **What to predict**: The repayment abilities
- **Submission file**:
For each SK_ID_CURR in the test set, you must predict a probability for the TARGET variable. The file should contain a header and have the following format:
```SK_ID_CURR,TARGET
100001,0.1
100005,0.9
100013,0.2
etc.
```
- **What kind of index value will the submitted items be evaluated?**: Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

### Problem 2
Learning and verification

In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score

# loading the csv of the dataset
df = pd.read_csv('application_train.csv')

# cleaning the dataset by removing the empy data(null)
cleaned_df = df.dropna()

# # separating them into variables
X = cleaned_df.loc[:,["AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY"]]
y = cleaned_df['TARGET']

In [20]:
# splitting the data into training and testing data using train_test_split from sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# standardizing the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

# fitting the data
reg = LinearRegression().fit(X_train_trans, y_train)

# predicting
reg_pred = reg.predict(X_test_trans)

print("MSE:", mean_squared_error(y_true=y_test, y_pred=reg_pred))
print("ROC", roc_auc_score(y_test,reg_pred))

MSE: 0.052153197403226256
ROC 0.5938285747369815


**MSE** is very low which is a good indication